# Data Extract and upload for NADAbase

## Steps

1. Extract Episodes for the period from Communicare
2. Extract ATOM data
3. Match Assessments with Episodes
4. Generate the upload Survey.txt file.

In [1]:
# import os

import pandas as pd

# import numpy as np
import mylogger
# from nada_load import load_and_parse_csv
from data_config import EstablishmentID_Program
from utils.df_xtrct_prep import  load_and_parse_csv
from utils.fromstr import convert_format_datestr

logger = mylogger.get(__name__)
# # List of column names in the CSV
# column_names = ['ESTABLISHMENT IDENTIFIER', 'GEOGRAPHICAL LOCATION', 'PMSEpisodeID', 'PMSPersonID', 'DOB', 'DOB STATUS', 'SEX', 'COUNTRY OF BIRTH', 'INDIGENOUS STATUS', 'PREFERRED LANGUAGE', 'SOURCE OF INCOME', 'LIVING ARRANGEMENT', 'USUAL ACCOMMODATION', 'CLIENT TYPE', 'PRINCIPAL DRUG OF CONCERN', 'PDCSubstanceOfConcern', 'ILLICIT USE', 'METHOD OF USE PRINCIPAL DRUG', 'INJECTING DRUG USE', 'SETTING', 'CommencementDate', 'POSTCODE', 'SOURCE OF REFERRAL', 'MAIN SERVICE', 'EndDate', 'END REASON', 'REFERRAL TO ANOTHER SERVICE', 'FAMILY NAME', 'GIVEN NAME', 'MIDDLE NAME', 'TITLE', 'SLK', 'MEDICARE NUMBER', 'PROPERTY NAME', 'UNIT FLAT NUMBER', 'STREET NUMBER', 'STREET NAME', 'SUBURB']




## Episode Data

In [2]:
# Global variables
active_clients_start_date ='2023-07-01' 
active_clients_end_date = '2023-12-31'

start_dt = convert_format_datestr(active_clients_start_date,'%Y-%m-%d', '%d%m%Y')
end_dt = convert_format_datestr(active_clients_end_date,'%Y-%m-%d', '%d%m%Y')


ep_source_filename = f'DATS_AllPrograms_{start_dt}-{end_dt}' #'DATS_Annual_AllPrograms_FY22-23'
ep_datasource_file_path = f"./data/in/NSW_CSV/{ep_source_filename}.csv"

# extract_start_date = 20130501
# extract_end_date = 20231231

# fname = f"{extract_start_date}_{extract_end_date}" # TODO :forNaada



results_folder = "./data/out/"

In [3]:
# # List of columns we care about
columns_of_interest = ['ESTABLISHMENT IDENTIFIER', 'GEOGRAPHICAL LOCATION', 'EPISODE ID','PERSON ID', 'SPECIFY DRUG OF CONCERN', 'PRINCIPAL DRUG OF CONCERN', 'START DATE', 'END DATE', 'SLK']
rename_columns = {
    'SPECIFY DRUG OF CONCERN': 'PDCSubstanceOfConcern',
    'PRINCIPAL DRUG OF CONCERN': 'PDCCode',
    'START DATE': 'CommencementDate',
    'END DATE': 'EndDate',
    'EPISODE ID': 'PMSEpisodeID',
    'PERSON ID': 'PMSPersonID',    
}

In [4]:
# ep_df  = load_and_parse_episode_csvs(ep_datasource_file_path)
ep_df = load_and_parse_csv(ep_datasource_file_path, rename_columns, columns_of_interest, date_cols=['START DATE', 'END DATE'])
ep_df['Program'] = ep_df['ESTABLISHMENT IDENTIFIER'].map(EstablishmentID_Program)

In [ ]:
# ep_df.head(10)

In [25]:
len(ep_df), min(ep_df.CommencementDate), max(ep_df.CommencementDate)

(787, datetime.date(2016, 11, 11), datetime.date(2023, 12, 22))

In [5]:
# ep_df['Program'] = ep_df['ESTABLISHMENT IDENTIFIER'].map(EstablishmentID_Program)
ep_df.rename(columns={'ESTABLISHMENT IDENTIFIER': 'AgencyCode'}, inplace=True)

In [7]:
ep_df.columns

Index(['AgencyCode', 'GEOGRAPHICAL LOCATION', 'PMSEpisodeID', 'PMSPersonID',
       'PDCCode', 'PDCSubstanceOfConcern', 'CommencementDate', 'EndDate',
       'SLK', 'Program'],
      dtype='object')

In [ ]:
# ep_df.head()

## ATOM Data


In [6]:
from utils.environment import MyEnvironmentConfig
from utils.df_xtrct_prep import extract_prep_atom_data


In [7]:
# to adjust the ODC parser
extract_start_date = 20230701
extract_end_date = 20231231
active_clients_start_date ='2023-10-01' 
active_clients_end_date = '2023-12-31'
fname = f"{extract_start_date}-{extract_end_date}" # TODO :forNaada
MyEnvironmentConfig().setup('prod')

processed_df = extract_prep_atom_data(extract_start_date, extract_end_date
                                 , active_clients_start_date
                                 , active_clients_end_date
                                 , fname, purpose='NADA')

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\Python311\Lib\logging\__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python311\Lib\logging\__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python311\Lib\logging\__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python311\Lib\logging\__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\aftab.jalal\dev\atom-analysis\.venv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\aftab.jalal\dev\at

In [ ]:
# processed_df = ['SLK',	'RowKey','ClientType',	'AssessmentDate', PDCSubstanceOrGambling	PDCMethodOfUse	PDCDaysInLast28	PDCUnits	PDCHowMuchPerOccasion	PDCAgeLastUsed	PDCGoals	PDC]
processed_df.columns

In [10]:
out = processed_df


In [12]:
out[ [c for c in out.columns if 'Cannabis' in c]] 

,Cannabis_TypicalQtyStr,Cannabis_DaysInLast28
448,NaN,NaN
593,NaN,NaN
580,NaN,NaN
548,NaN,NaN
472,0,0
...,...,...
1104,NaN,NaN
817,3.0; cones / joints,0
629,NaN,NaN
1164,0.5; grams,4


In [7]:
processed_df.K10Q12.value_counts()

K10Q12
18    8
25    3
21    2
1     2
5     2
10    2
15    2
2     2
8     2
13    1
28    1
14    1
3     1
20    1
23    1
Name: count, dtype: int64

In [11]:
# from utils.io import write_parquet
# processed_filepath = f"./data/processed/atom_NADA_{fname}.parquet"
# write_parquet(processed_df, processed_filepath) # don't force overwrite
processed_df.to_csv(f"./data/processed/atom_NADA_{fname}.csv")
# # ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column K10Q12 with type object')

In [8]:
atom_df  = processed_df.copy()
atom_df

,Program,RowKey,SLK,AssessmentType,Staff,AssessmentDate,SDSIsAODUseOutOfControl,SDSDoesMissingFixMakeAnxious,SDSHowMuchDoYouWorryAboutAODUse,SDSDoYouWishToStop,...,PrimaryCaregiver_0-5,PrimaryCaregiver_5-15,Past4WkBeenArrested,PaidWorkDays,StudyDays,Past4Wk_ViolentToYou,Past4WkHaveYouViolenceAbusive,Past4WkMentalHealth,Past4WkPhysicalHealth,Past4WkQualityOfLifeScore
472,EUROPATH,EUROPATH_INAS_20230703,EWNIM060119762,InitialAssessment,Paul.Crowe,2023-07-03,0.0,0.0,0.0,0.0,...,1,1,0,None,None,0,0,8.0,7,8.0
1145,MURMICE,MURMICE_ITSP_20230703,SACAR270619511,ITSPReview,Tina.Raccanello,2023-07-03,0.0,0.0,0.0,1.0,...,None,None,0,1,None,0,0,8.0,8,9.0
325,COCO,COCO_SITK_20230703,CONIN140619882,SupplementaryIntakeAssessment,Jim.Downey,2023-07-03,2.0,2.0,2.0,3.0,...,None,None,1,None,None,0,0,5.0,7,7.0
201,TSS,TSS_INAS_20230704,ASAHM240719861,InitialAssessment,Mimi.Avila,2023-07-04,2.0,0.0,0.0,2.0,...,1,1,1,None,None,0,0,5.0,7,5.0
527,TSS,TSS_ITSP_20230704,HEWOE211119841,ITSPReview,Maurice.Rickard,2023-07-04,1.0,1.0,2.0,2.0,...,None,None,0,20,0,0,0,7.0,6,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,ALONGSIDE,ALONGSIDE_ITSP_20231222,RIGNN170519632,ITSPReview,Lexxie.Jury,2023-12-22,3.0,2.0,2.0,3.0,...,None,None,0,0,0,0,0,7.0,4,7.0
817,SAPPHIRE,SAPPHIRE_INAS_20231222,ODEMI271219932,InitialAssessment,Kylie.Wood,2023-12-22,2.0,3.0,3.0,3.0,...,0,0,0,0,None,0,0,1.0,8,3.0
629,TSS,TSS_ITSP_20231222,ILION150419561,ITSPReview,Emiltus.Emmanuel,2023-12-22,0.0,0.0,0.0,0.0,...,None,None,0,None,None,0,0,9.0,9,8.0
1164,GOLBGNRL,GOLBGNRL_ITSP_20231222,TEARO220820081,ITSPReview,James.Costello,2023-12-22,0.0,1.0,1.0,1.0,...,None,None,0,None,None,0,0,6.0,9,8.0


In [14]:
# atom_df = atom_df.loc[ (atom_df.AssessmentDate>= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)]

In [12]:
atom_df['AssessmentDate'].info()  #describe()

<class 'pandas.core.series.Series'>
Index: 1319 entries, 448 to 799
Series name: AssessmentDate
Non-Null Count  Dtype         
--------------  -----         
1319 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 20.6 KB


In [10]:
# import pandas as pd
# # Ensure that AssessmentDate is in datetime format
# atom_df['AssessmentDate'] = pd.to_datetime(atom_df['AssessmentDate'], errors='coerce')

In [9]:
# atom_df.drop(atom_df[atom_df.Program.isin(['TSS', 'ARCA', 'COCO', 'PSYNSW' ])].index, inplace=True)
non_nsw_programs = ['TSS', 'ARCA', 'COCO', 'PSYNSW' ]
atom_df = atom_df[~atom_df['Program'].isin(non_nsw_programs)] 


In [14]:
len(ep_df), len(atom_df) 

(787, 546)

In [15]:
atom_df.Program.value_counts()

Program
SAPPHIRE     157
MURMICE      136
EUROPATH     127
GOLBICE      114
GOLBGNRL      66
MONPATH       55
MURMWIO       51
MURMPP        33
BUTTITOUT     30
ALONGSIDE     23
BEGAPATH      16
BegaMHHub     10
Name: count, dtype: int64

## NADA matching

In [10]:
# from match_audit import match_assessments
from data_prep import get_stage_per_episode
matching_ndays_slack = 7

In [30]:
# def match_assessments(episodes_df, atoms_df): #pdc_substance_mapping
#     # Apply the mapping to the ESTABLISHMENT IDENTIFIER and PDCSubstanceOfConcern columns in episodes_df
#     # episodes_df['ESTABLISHMENT IDENTIFIER'] = episodes_df['ESTABLISHMENT IDENTIFIER'].map(establishment_program_mapping)
#     # episodes_df['PDCSubstanceOfConcern'] = episodes_df['PDCSubstanceOfConcern'].map(pdc_substance_mapping)

#     # Merge the dataframes on SLK and Program
#     merged_df = pd.merge(episodes_df, atoms_df, how='inner', left_on=['SLK', 'Program'], right_on=['SLK', 'Program'])

#     # Filter rows where AssessmentDate falls within CommencementDate and EndDate (or after CommencementDate if EndDate is NaN)
#     matched_df = merged_df.loc[((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) & 
#                                 (merged_df['AssessmentDate'] <= merged_df['EndDate'])) |
#                                ((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) & 
#                                 (merged_df['EndDate'].isna()))]

#     # Check if PDCSubstanceOfConcern matches
#     # mismatched_df = matched_df.loc[matched_df['PDCSubstanceOfConcern_x'] != matched_df['PDCSubstanceOfConcern_y']]

#     # if len(mismatched_df) > 0:
#     #     logger.info(f"There are {len(mismatched_df)} rows where PDCSubstanceOfConcern does not match.")
#     #     logger.info(mismatched_df)

#     return matched_df

In [11]:
def get_mask_datefit(row, slack_days=7):
    # Convert to datetime if not already in that format
    # assessment_date = pd.to_datetime(row['AssessmentDate'], errors='coerce')
    # commencement_date = pd.to_datetime(row['CommencementDate'], errors='coerce')
    # end_date = pd.to_datetime(row['EndDate'], errors='coerce')

    # Create a Timedelta for slack days
    slack_td = pd.Timedelta(days=slack_days)

    # Check conditions
    after_commencement = row['AssessmentDate'].date() >= (row['CommencementDate'] - slack_td)
    before_end_date = row['AssessmentDate'].date() <= (row['EndDate'] + slack_td)

    return after_commencement and before_end_date


def match_assessments(episodes_df, atoms_df, matching_ndays_slack: int):

    # Merge the dataframes on SLK and Program
    df = pd.merge(episodes_df, atoms_df, how='inner', left_on=[
                  'SLK', 'Program'], right_on=['SLK', 'Program'])

    # Filter rows where AssessmentDate falls within CommencementDate and EndDate (or after CommencementDate if EndDate is NaN)
    mask = df.apply(get_mask_datefit, slack_days=matching_ndays_slack, axis=1)
    # mask = get_mask_datefit(df['AssessmentDate'], df['CommencementDate'],
    #                         df['EndDate'], slack_days=matching_ndays_slack)
    filtered_df = df[mask]
    # matched_df = merged_df.loc[((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) &
    #                             (merged_df['AssessmentDate'] <= merged_df['EndDate'])) |
    #                            ((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) &
    #                             (merged_df['EndDate'].isna()))]

    # Check if PDCSubstanceOfConcern matches
    # mismatched_df = matched_df.loc[matched_df['PDCSubstanceOfConcern_x'] != matched_df['PDCSubstanceOfConcern_y']]

    # if len(mismatched_df) > 0:
    #     logger.info(f"There are {len(mismatched_df)} rows where PDCSubstanceOfConcern does not match.")
    #     logger.info(mismatched_df)

    return filtered_df

In [12]:

matched_df = match_assessments(ep_df, atom_df, matching_ndays_slack)

In [13]:
# matched_df[['SLK', 'CommencementDate', 'AssessmentDate', 'EndDate']].head(30)
matched_df

,AgencyCode,GEOGRAPHICAL LOCATION,PMSEpisodeID,PMSPersonID,PDCCode,PDCSubstanceOfConcern,CommencementDate,EndDate,SLK,Program,...,PrimaryCaregiver_0-5,PrimaryCaregiver_5-15,Past4WkBeenArrested,PaidWorkDays,StudyDays,Past4Wk_ViolentToYou,Past4WkHaveYouViolenceAbusive,Past4WkMentalHealth,Past4WkPhysicalHealth,Past4WkQualityOfLifeScore
0,12QQ03076,10550,51655,8876,2101,Alcohol,2022-09-08,2023-11-28,RO2AC191120001,SAPPHIRE,...,None,None,0,0,None,0,1,2.0,5,4.0
1,12QQ03076,10550,51655,8876,2101,Alcohol,2022-09-08,2023-11-28,RO2AC191120001,SAPPHIRE,...,None,None,0,0,None,0,1,2.0,5,4.0
2,12QQ03076,10550,51655,8876,2101,Alcohol,2022-09-08,2023-11-28,RO2AC191120001,SAPPHIRE,...,None,None,0,13,None,0,1,6.0,8,5.0
3,12QQ03076,10550,51318,7331,2101,Alcohol,2022-09-06,2023-12-20,RAHAR221019911,SAPPHIRE,...,None,None,0,6,None,None,0,5.0,5,6.0
4,12QQ03076,10550,51318,7331,2101,Alcohol,2022-09-06,2023-12-20,RAHAR221019911,SAPPHIRE,...,None,None,0,6,None,None,0,6.0,5,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,12KK03024,17751,83777,9737,2101,Alcohol,2023-08-18,2024-01-14,URAEN170719911,MURMPP,...,None,None,0,28,None,None,0,7.0,9,8.0
511,12KK03024,17751,89462,9893,3103,Methamphetamine,2023-10-03,2023-12-13,HOPOB040719811,MURMPP,...,0,0,0,None,None,0,0,2.0,7,3.0
512,12KK03024,17751,93785,9322,3103,Methamphetamine,2023-11-07,2024-01-14,RAFRA030519901,MURMPP,...,None,None,0,20,None,None,0,8.0,8,8.0
513,12KK03024,17751,97444,2183,7101,Cannabinoids,2023-12-07,2024-01-14,AR2EL091019852,MURMPP,...,None,None,0,None,None,0,0,7.0,4,6.0


In [39]:
# PDCSubstanceOrGambling atom_df.columns
# ep_df.columns # PDCSubstanceOfConcern # PDCCode <- from episode

# matched_df.PDC <- from ATOM
# matched_df.PDCSubstanceOfConcern <- from Episode

0              Alcohol
1              Alcohol
2              Alcohol
3              Alcohol
4              Alcohol
            ...       
738            Alcohol
739    Methamphetamine
740    Methamphetamine
741       Cannabinoids
742    Methamphetamine
Name: PDCSubstanceOfConcern, Length: 690, dtype: object

In [ ]:
# list(matched_df.PDC.unique())
# list(ep_df.PDCSubstanceOfConcern.unique())
ep_df.PDCSubstanceOfConcern.value_counts()

In [46]:
# import numpy as np

# conditions = [
#     # matched_df['PDCSubstanceOfConcern'].isin(['Ethanol', 'Alcohols, n.e.c.']),
#      matched_df['PDCSubstanceOfConcern'].isin(['Alcohol']),
#     matched_df['PDCCode'].between(7100, 7199),
#     matched_df['PDCCode'].between(3100, 3199),
#     matched_df['PDCSubstanceOfConcern'].isin(['Pharmaceutical Opioids, n.f.d.']),
#     matched_df['PDCCode'].between(2400, 2499),
#     matched_df['PDCCode'] == 1202,
#     matched_df['PDCCode'].between(1100, 1399),
#     ~matched_df['PDCSubstanceOfConcern'].isin(['Nicotine', 'Cocaine'])
# ]
# # 'Alcohol', 'Cannabinoids and Related Drugs, nec', 'Heroin',
# #        'Cannabinoids', 'Benzodiazepines, nec', 'Methamphetamine',
# #        'Nicotine/tobacco', 'Cocaine', nan, 'Dexamphetamine',
# #        'Amphetamine', 'Other Volatile Solvents, nec', 'MDMA/Ecstasy',
# #        'Gambling', 'Alprazolam', 'Cannabinoid agonists', 'Buprenorphine'

# choices = [
#     'Alcohol',
#     'Cannabis',
#     'Amphetamines',
#     'Other Opioid Use',
#     'Tranquilisers',
#     'Heroin',
#     'Other Opioid Use',
#     'Another drug use'
# ]

# matched_df['PDCFinal'] = np.select(conditions, choices, default=matched_df['PDCSubstanceOfConcern'])

In [14]:
# Since the primary key in episodes_df is PMSEpisodeID, and in atoms_df it is a combination of SLK and Program, 
# you can use the groupby method on the merged_df to group by SLK and Program, 
# and then check if there is more than one unique PMSEpisodeID for any group.

# grouped = matched_df.groupby(['SLK', 'Program'])['PMSEpisodeID'].nunique()
grouped = matched_df.groupby(['SLK', 'RowKey'])['PMSEpisodeID'].nunique()
duplicates = grouped[grouped > 1]


In [15]:
duplicates

SLK             RowKey                
AHYAT141219641  SAPPHIRE_ITSP_20231128    2
Name: PMSEpisodeID, dtype: int64

In [32]:
# SLK + Program
onedupe = matched_df.loc[(matched_df.SLK =='AHYAT141219641') & (matched_df.Program == 'SAPPHIRE')]

In [29]:
onedupe [['Program', 'SLK', 'CommencementDate', 'AssessmentDate', 'EndDate', 'PMSEpisodeID', 'RowKey']]

NameError: name 'onedupe' is not defined

In [34]:

# Get the keys for the duplicate rows
duplicate_keys = duplicates.index

# Filter matched_df to show only rows that match the duplicate keys
duplicate_rows_df = matched_df[matched_df.set_index(['SLK', 'RowKey']).index.isin(duplicate_keys)]

# Show all rows in the resulting DataFrame
# pd.set_option('display.max_rows', None)  # Set this to display all rows



In [35]:
duplicate_rows_df[['SLK','RowKey',  'Program'  ,  'CommencementDate',   'EndDate' , 'PMSEpisodeID', 'AssessmentDate' ]]

,SLK,RowKey,Program,CommencementDate,EndDate,PMSEpisodeID,AssessmentDate
65,AHYAT141219641,SAPPHIRE_ITSP_20231128,SAPPHIRE,2023-07-11,2023-11-27,79760,2023-11-28
68,AHYAT141219641,SAPPHIRE_ITSP_20231128,SAPPHIRE,2023-11-28,2024-01-08,99200,2023-11-28


In [24]:
matched_df['Program'].value_counts()

Program
SAPPHIRE    109
MURMICE      89
EUROPATH     78
GOLBICE      73
GOLBGNRL     42
MONPATH      38
MURMWIO      24
MURMPP       17
BEGAPATH      8
Name: count, dtype: int64

In [43]:
# matched_df.to_csv(f"{results_folder}matched_df.csv", index=False)

In [38]:
matched_df.PMSEpisodeID.unique().shape

(510,)

In [ ]:
# atom_df[ (atom_df.AssessmentDate >= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)].count()

In [32]:
# len(atom_df[ (atom_df.AssessmentDate >= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)])

818

In [23]:
atom_df[atom_df['SLK'] =='OLOAC010820061'][[c for c in atom_df.columns if 'Nicotine_' in c]]

,Nicotine_TypicalQtyStr,Nicotine_DaysInLast28
868,,28
869,,28


In [ ]:
# len(matched_df.PMSEpisodeID.unique())

In [25]:
matched_df.to_csv('./data/out/NADA_Matched_20230701-2023-12-31.csv')

In [40]:
# df = pd.read_csv('./data/out/NADA_Matched_20230701-2023-12-31.csv')

In [17]:
# matched_df['Past4WkEngagedInOtheractivities.Paid Work'][0:2].apply(lambda x : x['Days'] if 'Days' in x)
matched_df.columns

Index(['AgencyCode', 'GEOGRAPHICAL LOCATION', 'PMSEpisodeID', 'PMSPersonID',
       'PDCCode', 'PDCSubstanceOfConcern', 'CommencementDate', 'EndDate',
       'SLK', 'Program', 'RowKey', 'AssessmentType', 'Staff', 'AssessmentDate',
       'SDSIsAODUseOutOfControl', 'SDSDoesMissingFixMakeAnxious',
       'SDSHowMuchDoYouWorryAboutAODUse', 'SDSDoYouWishToStop',
       'SDSHowDifficultToStopOrGoWithout', 'K10Q01', 'K10Q02', 'K10Q03',
       'K10Q04', 'K10Q05', 'K10Q06', 'K10Q07', 'K10Q08', 'K10Q09', 'K10Q10',
       'K10Q11', 'K10Q12', 'K10Q13', 'K10Q14', 'Alcohol_TypicalQtyStr',
       'Alcohol_DaysInLast28', 'Alcohol_PerOccassionUse',
       'Cannabis_TypicalQtyStr', 'Cannabis_DaysInLast28',
       'Heroin_TypicalQtyStr', 'Heroin_DaysInLast28',
       'Other Opioids_TypicalQtyStr', 'Other Opioids_DaysInLast28',
       'Cocaine_TypicalQtyStr', 'Cocaine_DaysInLast28',
       'Amphetamines_TypicalQtyStr', 'Amphetamines_DaysInLast28',
       'Benzodiazepines_TypicalQtyStr', 'Benzodiazepines_

In [18]:
from data_config import nada_final_fields


In [46]:
# rename = {'ESTABLISHMENT IDENTIFIER' : 'AgencyCode', } 
# matched_df.rename(columns=rename, inplace=True)



In [24]:
matched_df['AgencyCode'].value_counts()

AgencyCode
12QQ03076    109
13K034        89
12QQ03062     78
13Q035        73
12QQ03022     42
12QQ03061     38
12KK03025     24
12KK03024     17
12QQ03063      8
Name: count, dtype: int64

In [27]:
matched_df["Stage"] = get_stage_per_episode(matched_df)

In [26]:
min(matched_df['AssessmentDate']), max(matched_df['AssessmentDate'])

('01082023', '31102023')

In [21]:
matched_df['AssessmentDate'] = matched_df['AssessmentDate'].dt.strftime('%d%m%Y')


In [25]:
matched_df['ATOPHomeless']

0         0
1         0
2         0
3         0
4      None
       ... 
756       0
757       0
758    None
759       0
760    None
Name: ATOPHomeless, Length: 709, dtype: object

In [28]:
df_final = pd.DataFrame(columns=nada_final_fields)

for column in nada_final_fields:
    if column in matched_df.columns:
        df_final[column] = matched_df[column]  # Or use another default value
    else:
        df_final[column] =""


In [ ]:
# from utils.df_ops_base import drop_fields
# debug_cols_to_drop =['GEOGRAPHICAL LOCATION', 'PDCSubstanceOfConcern','CommencementDate' ,'EndDate','SLK','Program','RowKey', 'Staff','AssessmentType']
# nada_final_df = drop_fields(df_final,debug_cols_to_drop)

In [35]:
for mycol in df_final.columns:
  if not mycol in nada_final_fields:
    print(mycol)

In [ ]:
# ddmmyyyy - assesment date
# stage

In [36]:
df_final.columns

Index(['AgencyCode', 'PMSEpisodeID', 'PMSPersonID', 'Stage', 'AssessmentDate',
       'PDCCode', 'SDSIsAODUseOutOfControl', 'SDSDoesMissingFixMakeAnxious',
       'SDSHowMuchDoYouWorryAboutAODUse', 'SDSDoYouWishToStop',
       ...
       'ATOPHomeless', 'ATOPRiskEviction', 'PrimaryCaregiver_0-5',
       'PrimaryCaregiver_5-15', 'Past4WkBeenArrested', 'Past4Wk_ViolentToYou',
       'Past4WkHaveYouViolenceAbusive', 'Past4WkMentalHealth',
       'Past4WkPhysicalHealth', 'Past4WkQualityOfLifeScore'],
      dtype='object', length=160)

array(['12QQ03076', '12QQ03063', '12QQ03061', '12QQ03062', '12QQ03022',
       '13Q035', '13K034', '12KK03025', '12KK03024'], dtype=object)

In [34]:
survy_text_fname = f"{active_clients_start_date}_{active_clients_end_date}"

In [37]:
df_final['PMSPersonID'] = df_final['PMSPersonID'].astype(str).apply(lambda x: x.zfill(4))


In [39]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, 0 to 514
Columns: 160 entries, AgencyCode to Past4WkQualityOfLifeScore
dtypes: float64(7), int64(4), object(149)
memory usage: 601.2+ KB


In [41]:
float_cols = df_final.select_dtypes(include=['float']).columns
df_final[float_cols] = df_final[float_cols].astype('Int64')
# float_cols = df.select_dtypes(include=['float']).columns
# df[float_cols] = df[float_cols].astype('Int64')

In [42]:
my_agency_codes = df_final['AgencyCode'].unique()

for agency_code, program in EstablishmentID_Program.items():
  if not agency_code in my_agency_codes:
    continue
  out_path = f"./data/out/no-header/surveyTXT_{program}_{survy_text_fname}.csv"
  df_final[df_final.AgencyCode == agency_code].to_csv(out_path,index=False,header=False)
  out_path = f"./data/out/surveyTXT_{program}_{survy_text_fname}.csv"
  df_final[df_final.AgencyCode == agency_code].to_csv(out_path,index=False)
  

In [32]:
fname

'20230701-20231231'

# stats

In [31]:
df_final['AgencyCode'].value_counts()

AgencyCode
12QQ03076    109
13K034        89
12QQ03062     78
13Q035        73
12QQ03022     42
12QQ03061     38
12KK03025     24
12KK03024     17
12QQ03063      8
Name: count, dtype: int64

In [ ]:

# my_agency_codes = df_final['AgencyCode'].unique()
# for agency_code, program in EstablishmentID_Program.items():
#   if not agency_code in my_agency_codes:
#     continue
#   print(df_final['AgencyCode'].)

In [38]:
program_mask = (atom_df['Program'] =='SAPPHIRE') 
mask =  (atom_df['Benzodiazepines_DaysInLast28'].notna()) & (atom_df['Benzodiazepines_DaysInLast28'] != "") 
fields = [c for c in atom_df.columns if 'Benzodiazepines' in c]
fields.append('Program')
atom_df[program_mask & mask][  fields]

,Benzodiazepines_TypicalQtyStr,Benzodiazepines_DaysInLast28,Program
1033,10.0; dosage (mgs),28,SAPPHIRE
638,17.0; dosage (mgs),28,SAPPHIRE
639,17.0; dosage (mgs),28,SAPPHIRE
640,17.0; dosage (mgs),28,SAPPHIRE
1255,1.0; pills,14,SAPPHIRE
1042,0.5; dosage (mgs),8,SAPPHIRE
1034,2.0; dosage (mgs),28,SAPPHIRE


In [32]:
# df_final.to_csv("./data/out/surveyTXT_20230701-2023-12-31.csv")